# Code for Evaluating the LLM Models

In [1]:
#Imports
import time
import re, json
import pandas as pd
from llm import getChatChain
from app import load_documents_into_database
from langchain_community.llms import Ollama
from langchain.evaluation import load_evaluator
from langchain_community.vectorstores import Chroma

In [2]:
def extract_rating(evaluation: str) -> int:
    """
    Extracts the rating number from the evaluation string.
    It handles both single and double-bracketed formats.
    """
    match = re.search(r'Rating: \[?\[?(\d+)\]?\]?', evaluation)
    if match:
        return int(match.group(1))
    else:
        print(f"Invalid format for evaluation output: {evaluation}")
        return None

def evaluate_mistral(llm_model_name: str, db: Chroma) -> tuple:
    evaluator = load_evaluator("labeled_score_string", criteria="correctness", llm=Ollama(model=llm_model_name))
    chat = getChatChain(Ollama(model=llm_model_name), db)
    df = pd.read_csv("evaluate.csv")

    print("\n[INFO] Evaluating model: ", llm_model_name)
    
    f = open("Stats.csv", "a")
    
    for index, row in df.iterrows():
        inicio = time.time()

        question = row['question']
        reference_answer = row['answer']
        model_answer = chat(question=question)
        
        fim = time.time()
        tempo = round(float(fim - inicio), 2)

        try:
            evaluation = evaluator.evaluate_strings(
                prediction=model_answer,
                reference=reference_answer,
                input=question
            )
        except ValueError as e:
            print(f"ValueError: {e}")
            evaluation = str(e)  # Assign the exception message to evaluation

        # Debugging print statement to inspect the evaluation output
        print(f"Evaluation output: {evaluation}")

        # Convert the evaluation to a string if it is a dictionary
        if isinstance(evaluation, dict):
            evaluation = json.dumps(evaluation)

        # Extract the rating using the updated function
        score = extract_rating(evaluation)
        if score is None:
            continue

        # Debug print statements to confirm writing to the file
        print(f'\n[QUESTION] {question}')
        print(f'[SCORE] {score}')
        print(f'Writing to CSV: {llm_model_name},{score},{tempo}')
        f.write(f"{llm_model_name},{score},{tempo}\n")
        f.flush()  # Flush the file buffer to ensure data is written to disk

    f.close()  # Explicitly close the file
    print("Finished writing to CSV.")

In [3]:
def extract_rating(evaluation: str) -> int:
    """
    Extracts the rating number enclosed in single or double brackets from the evaluation string.
    """
    match = re.search(r'Rating: \[?\[?(\d+)\]?\]?', evaluation)
    if match:
        return int(match.group(1))
    else:
        print(f"Invalid format for evaluation output: {evaluation}")
        return None

def evaluate_llama2(llm_model_name: str, db: Chroma) -> tuple:
    evaluator = load_evaluator("labeled_score_string", criteria="correctness", llm=Ollama(model=llm_model_name))
    chat = getChatChain(Ollama(model=llm_model_name), db)
    df = pd.read_csv("evaluate.csv")

    print("\n[INFO] Evaluating model: ", llm_model_name)
    f = open("Stats.csv", "a")
    
    for index, row in df.iterrows():
        inicio = time.time()

        question = row['question']
        reference_answer = row['answer']
        model_answer = chat(question=question)
        
        fim = time.time()
        tempo = round(float(fim - inicio), 2)

        try:
            evaluation = evaluator.evaluate_strings(
                prediction=model_answer,
                reference=reference_answer,
                input=question
            )
        except ValueError as e:
            print(f"ValueError: {e}")
            evaluation = str(e)  # Assign the exception message to evaluation

        # Debugging print statement to inspect the evaluation output
        print(f"Evaluation output: {evaluation}")

        # Convert the evaluation to a string if it is a dictionary
        if isinstance(evaluation, dict):
            evaluation = json.dumps(evaluation)

        # Extract the rating using the new function
        score = extract_rating(evaluation)
        if score is None:
            continue

        print(f'\n[QUESTION] {question}')
        print(f'[SCORE] {score}')
        print(f'Writing to CSV: {llm_model_name},{score},{tempo}')
        f.write(f"{llm_model_name},{score},{tempo}\n")
        f.flush()  # Ensure data is written to disk

    f.close()  # Explicitly close the file
    print("Finished writing to CSV.")

In [4]:
def extract_rating(evaluation: str) -> int:
    """
    Extracts the rating number from the evaluation string.
    It handles both single and double-bracketed formats.
    """
    match = re.search(r'Rating: \[?\[?(\d+)\]?\]?', evaluation)
    if match:
        return int(match.group(1))
    else:
        print(f"Invalid format for evaluation output: {evaluation}")
        return None

def evaluate_zephyr(llm_model_name: str, db: Chroma) -> tuple:
    evaluator = load_evaluator("labeled_score_string", criteria="correctness", llm=Ollama(model=llm_model_name))
    chat = getChatChain(Ollama(model=llm_model_name), db)
    df = pd.read_csv("evaluate.csv")

    print("\n[INFO] Evaluating model: ", llm_model_name)
    
    try:
        f = open("Stats.csv", "a")
    except Exception as e:
        print(f"Failed to open file Stats.csv: {e}")
        return

    for index, row in df.iterrows():
        inicio = time.time()

        question = row['question']
        reference_answer = row['answer']
        model_answer = chat(question=question)
        
        fim = time.time()
        tempo = round(float(fim - inicio), 2)

        try:
            evaluation = evaluator.evaluate_strings(
                prediction=model_answer,
                reference=reference_answer,
                input=question
            )
        except ValueError as e:
            print(f"ValueError: {e}")
            evaluation = str(e)  # Assign the exception message to evaluation

        # Debugging print statement to inspect the evaluation output
        print(f"Evaluation output: {evaluation}")

        # Convert the evaluation to a string if it is a dictionary
        if isinstance(evaluation, dict):
            evaluation = json.dumps(evaluation)

        # Extract the rating using the updated function
        score = extract_rating(evaluation)
        if score is None:
            continue

        # Debug print statements to confirm writing to the file
        print(f'\n[QUESTION] {question}')
        print(f'[SCORE] {score}')
        print(f'Writing to CSV: {llm_model_name},{score},{tempo}')
        
        try:
            f.write(f"{llm_model_name},{score},{tempo}\n")
            f.flush()  # Ensure data is written to disk immediately
        except Exception as e:
            print(f"Failed to write to file: {e}")

    try:
        f.close()  # Explicitly close the file
    except Exception as e:
        print(f"Failed to close file: {e}")

    print("Finished writing to CSV.")

# Mistral

In [5]:
# Avaliação do Mistral segundo o Tempo, a Precisão e a Accuracy.
inicio = time.time()
db = load_documents_into_database("mistral", "nomic-embed-text", "../Final PDF Files", True)
evaluate_mistral("mistral", db)
fim = time.time()
print("O Modelo demorou " + str(round((fim-inicio), 2)) + " segundos a gerar as respostas.")

Loading documents
Loading .pdf files


100%|██████████| 25/25 [00:04<00:00,  5.81it/s]


Loading .md files


0it [00:00, ?it/s]


Creating embeddings and loading documents into Chroma

[INFO] Evaluating model:  mistral
 Based on the provided research in the "Advanced Techniques.pdf" document, there is no explicit information about the suggested rest period between sets for promoting muscle growth. The document primarily focuses on the concept of periodization and how training variables such as execution speed and load intensity can influence training adaptations. Therefore, I cannot answer this question with certainty based on the given research.ValueError: Invalid output:  Rating: 3. The assistant's response was not directly answering the user's question about the recommended rest time for hypertrophy. Instead, it mentioned that there is no explicit information in the provided document and focused on periodization instead. While the assistant acknowledged that they cannot answer with certainty based on the given research, a more definitive statement regarding the common recommendation for a rest period between s

# Llama2

In [5]:
#Avaliação do Llama2 segundo o Tempo, a Precisão e a Accuracy.
inicio = time.time()
db = load_documents_into_database("llama2","nomic-embed-text","../Final PDF Files",True)
evaluate_llama2("llama2",db)
fim = time.time()
print("O Modelo demorou " + str(round((fim-inicio),2)) + " segundos a gerar as respostas.")

Loading documents
Loading .pdf files


100%|██████████| 25/25 [00:03<00:00,  6.27it/s]


Loading .md files


0it [00:00, ?it/s]


Creating embeddings and loading documents into Chroma

[INFO] Evaluating model:  llama2
The recommended rest time between sets for muscle hypertrophy varies depending on the specific training program and goals. Generally, it is recommended to have a rest period of 30-2 minutes between sets for muscle hypertrophy. However, some studies suggest that shorter rest periods (20-60 seconds) may be effective for increasing muscular endurance. It's important to note that the optimal rest time will depend on individual factors such as fitness level, age, and training experience. It is recommended to consult with a qualified fitness professional or medical professional to determine the best rest time for your specific needs and goals.ValueError: Invalid output: Rating: 8

The assistant's response provides a good overview of the general guidelines for rest time between sets for muscle hypertrophy. The response mentions that the recommended rest time varies depending on the specific training progra

# Zephyr

In [6]:
#Avaliação do Zephyr segundo o Tempo, a Precisão e a Accuracy.
inicio = time.time()
db = load_documents_into_database("zephyr","nomic-embed-text","../Final PDF Files",True)
evaluate_zephyr("zephyr",db)
fim = time.time()
print("O Modelo demorou " + str(round((fim-inicio),2)) + " segundos a gerar as respostas.")

Loading documents
Loading .pdf files


100%|██████████| 25/25 [00:03<00:00,  6.41it/s]


Loading .md files


0it [00:00, ?it/s]


Creating embeddings and loading documents into Chroma

[INFO] Evaluating model:  zephyr
Based on the provided research, if your goal is muscle mass, it's generally recommended that you should aim for a rest time of 1-2 minutes between sets. This allows for adequate recovery to promote muscular hypertrophy without compromising form and progress in your training program. However, individual factors such as fitness level and training program design may influence this recommendation. It's always best to consult with a healthcare professional or certified personal trainer for personalized advice on rest time based on your specific goals and needs.ValueError: Invalid output: Explanation:
The assistant's answer provides an appropriate response to the user's question. It accurately states that the recommended rest time for hypertrophy is typically 60-90 seconds, which falls within the guidelines provided in the ground truth. The assistant then expands on this information by providing additiona